# 支援模型上下文協議 (MCP) 的 Azure AI Agents

此筆記本展示如何使用支援模型上下文協議 (MCP) 工具的 Azure AI Agents，來建立一個能夠利用外部 MCP 伺服器增強功能的智能代理。


## 安裝所需的 NuGet 套件

首先，我們需要安裝 Azure AI Agents Persistent 套件，該套件提供使用 Azure AI Agents 的核心功能。


## 無密鑰身份驗證的好處

此筆記本展示了**無密鑰身份驗證**，提供以下幾個優勢：
- ✅ **無需管理 API 密鑰** - 使用基於 Azure 身份的身份驗證
- ✅ **增強安全性** - 無需在代碼或配置中存儲機密
- ✅ **自動憑證輪換** - Azure 負責憑證生命周期管理
- ✅ **基於角色的訪問控制** - 使用 Azure RBAC 提供細粒度的權限

`DefaultAzureCredential` 將自動使用最佳可用的憑證來源：
1. 託管身份（在 Azure 中運行時）
2. Azure CLI 憑證（開發期間）
3. Visual Studio 憑證
4. 環境變數（如果已配置）


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

安裝 Azure Identity 套件以使用 DefaultAzureCredential 驗證 Azure 服務。


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## 匯入所需的命名空間

匯入 Azure AI Agents 和 Azure Identity 所需的命名空間。


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## 配置 Azure AI Agent Client（無密鑰驗證）

設置配置變數並使用**無密鑰驗證**創建 PersistentAgentsClient：
- **projectEndpoint**：Azure AI Foundry 項目的端點
- **modelDeploymentName**：已部署 AI 模型的名稱（GPT-4.1 nano）
- **mcpServerUrl**：MCP 伺服器的 URL（Microsoft Learn API）
- **mcpServerLabel**：用於識別 MCP 伺服器的標籤
- **DefaultAzureCredential**：使用受管理的身份、Azure CLI 或其他憑證來源（無需 API 密鑰）


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## 建立 MCP 工具定義

建立一個 MCP 工具定義，連接到 Microsoft Learn MCP 伺服器。這將允許代理存取 Microsoft Learn 的內容和文件。


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## 建立 AI 代理

建立一個持久的 AI 代理，使用指定的模型和 MCP 工具。該代理配置如下：
- GPT-4.1 nano 模型
- 指示使用 MCP 工具進行協助
- 可存取 Microsoft Learn MCP 伺服器


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## 建立對話線索並發送訊息

建立一個對話線索，並發送用戶訊息詢問 Azure OpenAI 和 OpenAI 之間的差異。這將測試代理使用 MCP 工具提供準確資訊的能力。


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## 配置 MCP 工具資源（無密鑰）

設置 MCP 工具資源。若採用真正的無密鑰方法，並且 MCP 伺服器支援基於 Azure 身份驗證，則可以移除自定義標頭。以下範例展示了如何添加標頭（如有需要），但在無密鑰情境下，這些可能並非必要。


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## 啟動代理運行

建立並啟動一次運行以處理使用者的訊息。代理將使用已配置的 MCP 工具和資源來生成回應。


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## 監控執行狀態及處理工具批准（無密鑰）

監控代理執行狀態並處理任何所需的工具批准。此循環：
1. 等待執行完成或需要操作
2. 當需要時，自動批准 MCP 工具調用
3. 對於無密鑰身份驗證，如果 MCP 伺服器支援 Azure 身份，可能不需要標頭


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## 顯示對話結果

檢索並顯示線索中的所有訊息，展示用戶的問題以及代理的回應。訊息會按照時間順序排列，並附上時間戳和角色標示。


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**免責聲明**：  
本文件已使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於重要資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋概不負責。
